In [6]:
from math import pi as PI
import math
import torch
from torch import nn, optim
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import argparse
import os
import ast
import matplotlib.ticker as ticker

from PINN.diff import diff
from PINN.networks import FCNN
from PINN.conditions import BoundaryCondition
from PINN.generators import generator_2dspatial_rectangle, generator_2dspatial_segment
from PINN.solvers import SingleNetworkApproximator2DSpatial_deform,_solve_2dspatial
from PINN.monitors import Monitor2DSpatial_deform
from PINN.parameters import *



In [7]:
parser = argparse.ArgumentParser(description='PyTorch Training Heat Forward')

# 添加命令行参数
parser.add_argument('--lr', type=float, default=0.0002, help='Learning rate')
parser.add_argument('--batch_size', type=int, default=1024, help='Batch size')
parser.add_argument('--epochs', type=int, default=1000000, help='Total training epochs')
parser.add_argument('--gpu', type=bool , default=False ,help='If use GPU training')
parser.add_argument('--train_rec_size', type=int , default=128 ,help='Points generated in rectangle 128*128')
parser.add_argument('--train_gen_random', type=bool , default=True ,help='If random generate points')
# BRR-PINN
parser.add_argument('--weight_equ1', type=int , default=7 ,help='equation weight')
parser.add_argument('--weight_equ2', type=int , default=7 ,help='equation weight') 
parser.add_argument('--weight_equ3', type=int , default=15 ,help='equation weight')  
parser.add_argument('--weight_equ4', type=int , default=5 ,help='equation weight') 
parser.add_argument('--weight_equ5', type=int , default=15 ,help='equation weight') 
parser.add_argument('--weight_equ6', type=int , default=100 ,help='equation weight')

parser.add_argument('--brr', type=int , default=40 ,help=' BRR coefficient')
parser.add_argument('--center_value', type=int , default=1 ,help='Center brr coefficient')
parser.add_argument('--network_MLP', type=str , default="(128,128,128,128,128)" ,help='Network shape')
parser.add_argument('--check_every', type=int , default=1000 ,help='Check period')
parser.add_argument('--save_dict', type=str , default='train_result/brrpinn_force' ,help='Save dictionary')
parser.add_argument('--impose', type=int , default=1 ,help='If exactly impose boundary')



args = parser.parse_args(args=[])
#args = parser.parse_args() #If use .py file to train
print(args)

save_image_folder = args.save_dict + "-image/"
save_model_folder=args.save_dict + "-model/"

if not os.path.exists(save_image_folder):
    os.makedirs(save_image_folder)
if not os.path.exists(save_model_folder):
    os.makedirs(save_model_folder)

use_gpu = args.gpu
device = torch.device("cuda" if use_gpu else "cpu")
if use_gpu:
        cuda_kwargs = {'num_workers': 1,
                       'pin_memory': True,
                       'shuffle': True}

Namespace(lr=0.0002, batch_size=1024, epochs=1000000, gpu=False, train_rec_size=32, train_gen_random=True, weight_equ1=7, weight_equ2=7, weight_equ3=15, weight_equ4=5, weight_equ5=15, weight_equ6=100, brr=40, center_value=1, network_MLP='(128,128,128,128,128)', check_every=100, save_dict='train_result/brrpinn_force', impose=1)


In [8]:
def calculate_sigma_rr(u,w,r,z):
    return 2*G * ((1-mu)/(1-2*mu)*diff(u,r) + mu/(1-2*mu)*(u/r+diff(w,z))) *1e-6

def calculate_sigma_theta(u,w,r,z):
    return 2*G * ((1-mu)/(1-2*mu)*u/r + mu/(1-2*mu)*(diff(w,z)+diff(u,r)))*1e-6

def calculate_sigma_zz(u,w,r,z):
    return 2*G * ((1-mu)/(1-2*mu)*diff(w,z) + mu/(1-2*mu)*((u/r)+diff(u,r)))*1e-6

def calculate_tau_zr(u,w,r,z):
    return G*(diff(w,r)+diff(u,z)) *1e-6   # MPa  um

def force_balance_r(u,w,r,z):
    sigma_rr=calculate_sigma_rr(u,w,r,z)
    sigma_theta=calculate_sigma_theta(u,w,r,z)
    tau_zr=calculate_tau_zr(u,w,r,z)
    return diff(sigma_rr,r)+diff(tau_zr,z)+(sigma_rr-sigma_theta)/r

def force_balance_z(u,w,r,z):
    sigma_zz=calculate_sigma_zz(u,w,r,z)
    tau_zr=calculate_tau_zr(u,w,r,z)
    return diff(sigma_zz,z)+diff(tau_zr,r)+tau_zr/r



In [9]:
# Monitor metrics using to evaluate results during training
metrics={}

def equ1(uu,xx,yy):
    error=force_balance_r(uu[:,0],uu[:,1],xx,yy)
    # error=force_balance_r(uu,xx,yy)
    return torch.mean(abs(error)**2)
metrics['equ1']=equ1

def equ2(uu,xx,yy):
    error=force_balance_z(uu[:,0],uu[:,1],xx,yy)
    # error=force_balance_z(uu,xx,yy)
    return torch.mean(abs(error)**2)
metrics['equ2']=equ2

def equ3(uu,xx,yy):
    error=calculate_sigma_rr(uu[:,0],uu[:,1],xx,yy)-uu[:,2]
    return torch.mean(abs(error)**2)
metrics['equ3']=equ3

def equ4(uu,xx,yy):
    error=calculate_sigma_theta(uu[:,0],uu[:,1],xx,yy)-uu[:,3]
    return torch.mean(abs(error)**2)
metrics['equ4']=equ4

def equ5(uu,xx,yy):
    error=calculate_sigma_zz(uu[:,0],uu[:,1],xx,yy)-uu[:,4]
    return torch.mean(abs(error)**2)
metrics['equ5']=equ5

def equ6(uu,xx,yy):
    error=calculate_tau_zr(uu[:,0],uu[:,1],xx,yy)-uu[:,5]
    return torch.mean(abs(error)**2)
metrics['equ6']=equ6

def comsol_compare_u(uu,xx,yy):
    x=torch.linspace(0.0, 1.0, 400).requires_grad_()
    y=torch.linspace(0.0, 1.0, 400).requires_grad_()
    xy_tensor = torch.cartesian_prod(x, y).to(device)
    xx = torch.squeeze(xy_tensor[:, 0])
    yy = torch.squeeze(xy_tensor[:, 1])

    uu=fcnn_approximator.__call__(xx,yy)
    uu=uu.detach().cpu().numpy()
    fem=pd.read_csv('./fem_data/force.txt',delimiter=r'\s+')
    uu_di=abs(uu[:,0]-fem['u'].values)
    return np.mean(uu_di **2 )
metrics['comsol_compare_u']=comsol_compare_u

def comsol_compare_w(uu,xx,yy):
    x=torch.linspace(0.0, 1.0, 400).requires_grad_()
    y=torch.linspace(0.0, 1.0, 400).requires_grad_()
    xy_tensor = torch.cartesian_prod(x, y).to(device)
    xx = torch.squeeze(xy_tensor[:, 0])
    yy = torch.squeeze(xy_tensor[:, 1])

    uu=fcnn_approximator.__call__(xx,yy)
    uu=uu.detach().cpu().numpy()
    fem=pd.read_csv('./fem_data/force.txt',delimiter=r'\s+')
    uu_di=abs(uu[:,0]-fem['w'].values)
    return np.mean(uu_di **2 )
metrics['comsol_compare_w']=comsol_compare_w


fcnn = FCNN(
    n_input_units=2,
    n_output_units=6, # [u(r-direction deformation), w(z-direction deformation), sigma_rr, sigma_theta, sigma_zz, tau_zr]
    hidden_units=ast.literal_eval(args.network_MLP),
    actv=nn.Tanh
)

fcnn=fcnn.to(device)

fcnn_approximator = SingleNetworkApproximator2DSpatial_deform(
    single_network=fcnn,
    #single_network=renn,
    pde=(force_balance_r,force_balance_z,calculate_sigma_rr,\
         calculate_sigma_theta,calculate_sigma_zz,calculate_tau_zr),
    boundary_conditions=[],
    args=args
)

size_train=args.train_rec_size
adam=optim.Adam(fcnn_approximator.parameters(),lr=args.lr)
train_gen_spatial = generator_2dspatial_rectangle(size=(size_train, size_train), x_min=r1, x_max=r2, y_min=0.0, y_max=h1,device=device,random=args.train_gen_random,bound=True)

In [ ]:
#%matplotlib inline
heat_transfer_2d_solution, _ = _solve_2dspatial(
    train_generator_spatial=train_gen_spatial,
    approximator=fcnn_approximator,
    optimizer=adam,
    batch_size=args.batch_size,
    max_epochs=args.epochs,
    shuffle=False,
    metrics=metrics,
    monitor=Monitor2DSpatial_deform(        
        check_on_x=torch.linspace(r1, r2, 200),
        check_on_y=torch.linspace(0.0, h1, 200),
        check_every=args.check_every,
        device=device,
        args=args
    ),
    device=device,
    args=args
)